In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
from astropy.table import Table,vstack
from desiutil.log import get_logger, DEBUG
from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *

from timedomain.sp_utils import SkyPortal as sp
import requests
import datetime

In [ ]:
# Loop over unprocessed night, doesn't need to be run as we debug
night_arr = getUnprocessedDates()
for a in night_arr:   
    # read in and store in one place all the fibermap information in the spectra files
    date = a

In [ ]:
date=20210606
tile_petal,group_tid,group_tp, group_night =getMatchedTileid(date)

In [ ]:
#Set non-default plot size 
plt.rcParams["figure.figsize"] = (20,6)

#SkyPortal token:
token = "497850e8-c7b1-48de-b014-fc0e011f2319"

filter_name = 'DESIDIFF'

In [ ]:
# tile_petal contain subsets of tile/petals whose RA/DEC targets are not contained in other tile/petals from that night
for tps in tile_petal:
    # Check to see if this tile/petal has nothing to subtract.  Skip's IO if this is the case
    if hasNothingToProcess(tps,group_tid,group_tp,group_night):
        continue
    
    # cache spectra to minimize IO
    spectra_list=dict()
    zbest_list = []
    target_list=[]
    for tp in tps:
        filename =  f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{tp[0]}/{date}/spectra-{tp[1]}-{tp[0]}-thru{date}.fits"
        spectra_list[(tp[0],tp[1])]=read_spectra(filename)
        zbest = filename.replace('spectra','zbest')
        z = Table.read(zbest, format='fits',hdu=1, memmap=True)['TARGETID','Z','ZERR','ZWARN','SPECTYPE']
        zbest_list.append(z)
    z = vstack(zbest_list)
    

#     z_data = []
    count=0
    # loop over all unique RA/DEC pairs from that night
    for tid, tp, night in zip(group_tid,group_tp,group_night):
        # if this RA/DEC is not in thie tile_petal combination than skip
        if tp[0] not in tps:
            continue
            
        if len(night) == 1:
            continue
            
#         print(tid)

        # Eliminate those with no reference night here
        # Proceed with a subtraction for this object
        
        # The coadds of the new and reference are constructed from all spectra with all targetid's in tid and all
        # tile/petal combinations in tp, which are cached above
        
        # Finds the object in the zbest by matching the targetID.
        #Stores a tuple = (TARGETID, Z, ZERR,ZWARN, SPECTYPE)
        z_data = []
        for t in tid:
            tid_ind = list(z['TARGETID']).index(t)
            z_data.append((tid, z[tid_ind]['Z'], z[tid_ind]['ZERR'], z[tid_ind]['ZWARN'], z[tid_ind]['SPECTYPE']))
        
        newSpectra=[]
        refSpectra=[]
        for tile,plate in tp:
            spec = spectra_list[(tile,plate)]

            idx = numpy.in1d(night, date)
            ref_night = night[~idx]
            
            newSpectra.append(spec.select(nights=date, targets = tid))
            
            """
            There is a variable night that has all the nights associated with this RA/DEC
            derive the ref_night from that
            """
            
            refSpectra.append(spec.select(nights=ref_night, targets = tid))       
         
        
        newflux, newivar, newwave, newmask = coadd(newSpectra)
        refflux, refivar, refwave, refmask = coadd(refSpectra)
        
        difflux, difivar, difmask, difwave = dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"])
        
        difflux = {key: newflux[key] - refflux[key]
                       for key in newflux.keys()}
        difivar = {key: newivar[key] + refivar[key]
                       for key in newivar.keys()}
        difmask = {key: newmask[key] + refmask[key]
                       for key in newmask.keys()}
        difwave = dict(newwave)

        ## filters         
        # trim red edge 
        if 'z' in difflux.keys():
            w=numpy.where(difwave['z'] < 9100)[0]
            difflux['z']=difflux['z'][w]
            difivar['z']=difivar['z'][w]
            difwave['z']=difwave['z'][w]
            difmask['z']=difmask['z'][w]
            
        # mean-subtracted difference
        difflux_clipmean = clipmean(difflux,difivar,difmask)
        
        # Difference spectrum may have broadband signal
        perband_filter = perband_SN(difflux_clipmean,difivar,difmask)
        
        # fractional increase
        perband_inc = perband_increase(difflux_clipmean,difivar,difmask, refflux,refivar,refmask)
        
        # Difference spectrum may have high-frequency signal
        perres_filter = perconv_SN(difflux_clipmean,difivar,difmask)
        
#         print(perband_filter, perband_inc, perres_filter)
#         print(any(list(perband_filter.values())), numpy.sum(list(perres_filter.values())) >1)

        #broadband
        bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
        linelogic = numpy.sum(list(perres_filter.values()))>4
        logic = bblogic or linelogic
        plt.clf()
        if logic:
            #Uncomment next line if you want to print only those TargetIds that get plotted
            print(tid)
            for b in difflux.keys():
                w=numpy.where(difmask[b] ==0)[0]
                if b == list(difflux.keys())[-1]:
                    plt.plot(difwave[b][w],difflux[b][w],color='cyan', label = 'Difference')
                    plt.plot(newwave[b][w],newflux[b][w],color='blue',alpha=0.5, label = 'New Spectrum')
                    plt.plot(refwave[b][w],refflux[b][w],color='green',alpha=0.5, label = 'Reference Spectrum')
                    plt.legend()
                else:
                    plt.plot(difwave[b][w],difflux[b][w],color='cyan')
                    plt.plot(newwave[b][w],newflux[b][w],color='blue',alpha=0.5)
                    plt.plot(refwave[b][w],refflux[b][w],color='green',alpha=0.5)

            plt.xlabel('Wavelength (A)')
            plt.ylabel('Flux') 
            plt.show()
            count=count+1
            
            #SkyPortal functionality begins:
            #Combine the wavelengths into one 
            wavelengths = difwave['b'].tolist() + difwave['r'].tolist() + difwave['z'].tolist()
            
           #Combine the differenced flux into one 
            flux = difflux['b'].tolist() + difflux['r'].tolist() + difflux['z'].tolist()
            
            
            #SkyPortal's Id for the object
            objID = 'DESI{}'.format(str(tid[0]))
            
            #Code to check if this object already exists in SkyPortal. If not, create it
            response = requests.get("http://desi2.lbl.gov:5000/api/candidates/{}".format(objID), headers={"Authorization": f"token {token}"})
            if response.status_code == 400:
                obj_data = {
                        "ra": 0, #RA is required when creating a new object, I have currently set them all to 0, since it doesn't affect our purpose of uploading the spectrum. We should discuss this. 
                        "dec": 0, #Same for DEC
                        "id": objID,
                        "filter_ids": [sp.filter_id(filter_name)],
                        "passed_at": str(datetime.datetime.utcnow()) #UTC time when the object passed the filter
                        }
                sp.api('POST', '{}/api/candidates/obj_id'.format(sp.url), data = obj_data)
                
            #Now, we send the differenced spectrum to SkyPortal
            spectrum_data = {
                        "obj_id": objID,
                        "wavelengths": wavelengths,
                        "fluxes": flux,
                        "observed_at": str(datetime.datetime.utcnow()), #UTC time when the difference spectrum was created, same as filter-passing time
                        "instrument_id": sp.instrument_id()
                        }
            #First, check if the same spectrum already exists under the object
            response = requests.get("http://desi2.lbl.gov:5000/api/sources/{}/spectra".format(objID),headers={"Authorization": f"token {token}"})
            if (flux not in response.json()['data']['spectra'][0]['fluxes']): #only if the exact same spectrum doesn't already exist, upload it.
                sp.api('POST', '{}/api/spectrum'.format(sp.url), data = spectrum_data)
        if count==10:
            wefwe

In [70]:
print(response.json()['data']['spectra'][0]['fluxes'])

[1.0594978332519531, 1.0190545320510864, -11.739490270614624, -3.6231460571289062, -10.28920030593872, 6.644783973693848, -0.404541015625, 6.581346750259399, 6.585370063781738, 3.8816514015197754, -11.542001485824585, -2.371721029281616, 6.259595632553101, -3.2723641395568848, 7.395030498504639, 2.1202378273010254, 11.052502617239952, 11.471781849861145, -6.573673129081726, -1.001082420349121, -12.758077621459961, -2.425534248352051, -7.391916066408157, 2.3674373626708984, 12.097890377044678, 14.889042377471924, 5.91380500793457, 3.3221035599708557, -13.402461111545563, 1.4156131744384766, -1.7109053134918213, 6.601886197924614, 8.808484077453613, 1.801997184753418, 5.289763450622559, -0.1615772247314453, -2.679487705230713, 7.9037024974823, -6.932082891464233, 0.3235194683074951, 8.397032737731934, 2.8008298873901367, 3.964763343334198, 2.8640270233154297, 6.699763059616089, -1.295110821723938, -8.066623210906982, 3.7968063354492188, -1.6664795875549316, -3.5854501724243164, -6.513390